# Robust Knapsack

This tutorial shows how to construct and solve the Robust knapsack optimization problem (Bertsimas and Simm 2004, (https://pubsonline.informs.org/doi/abs/10.1287/opre.1030.0065). The problem can be formulated as follows:
$$
\begin{align}
\max_{\mathbf{x}} &\mathbf{c}^T\mathbf{x} \nonumber \\

\text{s.t. } &\left( \mathbf{w} + \mathbf{\Delta}\mathbf{z} \right)\mathbf{x} \leq b \nonumber \\

&\mathbf{x}\in \left\{ 0,1 \right\}^n \nonumber \\

&\mathbf{z}\in \left\{\mathbf{z}:  \left \lVert \mathbf{z} \right \rVert_{\infty} \leq 1, \left \lVert \mathbf{z} \right \rVert_{1} \leq r \right\} \nonumber \\
\end{align}
$$

where the items values are denoted by $\mathbf{c}$, and the uncertainty is represented by $\mathbf{z}$. The matrix $\Delta$ is a diagonal matrix. The vector $\mathbf{w}$ stores the nominal weight coefficients.

In [1]:
import cvxpy as cp
import numpy as np

from lropt import Budget
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

We define the constants as shown below:

In [2]:
N = 50
b = 2000

c = 2*np.random.randint(low=5, high=10, size=N)    # profit coefficients
w = 2*np.random.randint(low=10, high=41, size=N)   # nominal weights
delta = np.diag(0.2*w)                             # maximum deviations
r = 2                                              # uncertainty budget

The problem is defined and solved as shown below:

In [3]:
uncertainty_set = Budget(rho1=1, rho2=r)
x = cp.Variable(N, boolean=True)
z = UncertainParameter(N, uncertainty_set=uncertainty_set)

objective = cp.Maximize(c@x)
constraints = [
                (w + delta@z)@x <= b,
              ]
prob = RobustProblem(objective=objective, constraints=constraints)
prob.solve()